To look up routes from outside London, we need to specify the icscode of the station.  This script gathers icscodes for all stations

In [1]:
from mylibrary.connections import Automapped_Base, session
Stations = Automapped_Base.classes.all_stations

/Users/robinlinacre/anaconda/lib/python2.7/site-packages/sqlalchemy/dialects/postgresql/base.py:2505: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))


In [2]:
# ids = session.query(Stations.nlc).filter(Stations.london_or_gb == "gb").all()
nlcs = session.query(Stations.nlc).all()

In [3]:
from mylibrary.secrets import app_id, app_key

In [4]:
# Dump all the json to the database first.  Then process it later
# Iterate through adding icscode
import requests

from mylibrary.tfl_helpers import status_of_tfl_response_places

def get_icsCode(lat,lng):
    my_dict = {"lat": lat,
        "lng": lng,
        "id": app_id,
        "key": app_key}
    
    full_str = "".join([r"https://api.tfl.gov.uk/Place?",
    r"lat={lat}",
    r"&lon={lng}",
    r"&radius=1000",
    r"&includeChildren=False",
    r"&app_id={id}",
    r"&app_key={key}"])
    
    url = full_str.format(**my_dict)
    r = requests.get(url)
    
    message = status_of_tfl_response_places(r.content)
    
    return_object = {"json": r.content, "request_url": url, "tfl_message": message}
    return return_object
    

for nlc in nlcs:
    station = session.query(Stations).filter(Stations.nlc == nlc).one()
    ics_object = get_icsCode(station.lat, station.lng)
    station.tfl_request = ics_object["request_url"]
    station.tfl_response = ics_object["json"]
    station.tfl_message = ics_object["tfl_message"]
    session.add(station)
    session.commit()

/Users/robinlinacre/anaconda/lib/python2.7/site-packages/sqlalchemy/sql/type_api.py:279: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return x == y
